In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
mdi_df = pd.read_csv("Marvel_DC_imdb.csv")
mdi_df.head()

Unnamed: 0                              Movie      Year  \
0           0                           Eternals    (2021)   
1           1                               Loki  (2021– )   
2           2  The Falcon and the Winter Soldier    (2021)   
3           3                        WandaVision    (2021)   
4           4            Spider-Man: No Way Home    (2021)   

                      Genre  RunTime Rating   Director  \
0    Action,Adventure,Drama      NaN    NaN  ChloéZhao   
1  Action,Adventure,Fantasy      NaN    NaN        NaN   
2    Action,Adventure,Drama   50 min  TV-14        NaN   
3       Action,Comedy,Drama  350 min  TV-PG        NaN   
4   Action,Adventure,Sci-Fi      NaN    NaN   JonWatts   

                                               Actor  \
0  AngelinaJolie,GemmaChan,RichardMadden,BarryKeo...   
1  TomHiddleston,OwenWilson,SophiaDiMartino,Richa...   
2  AnthonyMackie,SebastianStan,WyattRussell,ErinK...   
3  ElizabethOlsen,PaulBettany,KathrynHahn,Teyonah...   
4        AngourieRice,TomHolland,Zendaya,MarisaTomei   

                                         Description  IMDB_Score  Metascore  \
0  The saga of the Eternals, a race of immortal b...         NaN        NaN   
1      A new Marvel chapter with Loki at its center.         NaN        NaN   
2  Following the events of 'Avengers: Endgame,' S...         7.5        NaN   
3  Blends the style of classic sitcoms with the M...         8.1        NaN   
4       A continuation of Spider-Man: Far From Home.         NaN        NaN   

     Votes USA_Gross Category  
0      NaN       NaN   Marvel  
1      NaN       NaN   Marvel  
2  105,557       NaN   Marvel  
3  174,710       NaN   Marvel  
4      NaN       NaN   Marvel

## Khám phá dữ liệu

### Các thông tin về bộ dữ liệu

Đây là một bộ dữ liệu về một số bộ phim của Marvel và DC. Trong bộ dữ liệu này, ta có các cột như sau:

1. **Unnamed**: index sẵn có từ dataset.
2. **Movie**: tên bộ phim.
3. **Year**: (các) năm thực hiện bộ phim.
4. **Genre**: thể loại.
5. **Runtime**: thời lượng phim.
6. **Rating**: Phân loại đối tượng xem phim theo tiêu chuẩn của MPAA (Motion Picture Association of America - Hiệp hội Điện ảnh Hoa Kỳ).
7. **Director**: đạo diễn phim.
8. **Actor**: Các diễn viên tham gia.
9. **Description**: mô tả tóm tắt về nội dung phim.
10. **IMDB_score**: điểm đánh giá từ IMDb (Internet Movie Database), một thư viện về phim điện ảnh nổi tiếng được nhiều người lựa chọn. Những bộ phim hay thường có IMDB score cao.
11. **Metascore**: điểm đánh giá từ các nhà phê bình từ Metacritic, một trang tổng hợp các bài đánh giá về album nhạc, trò chơi điện tử, phim điện ảnh, chương trình truyền hình và đĩa DVD.
12. **Votes**: số lượt bình chọn cho phim từ các khán giả.
13. **USA_gross**: doanh thu của phim.
14. **Category**: hãng làm phim (Marvel hoặc DC).

In [3]:
mdi_df.shape

(1690, 14)

Các giá trị trống được điền mặc định là NaN, ta kiểm tra mức độ dữ liệu bị thiếu trong bộ dữ liệu này.

In [4]:
round(mdi_df.isna().mean(), 2)

Unnamed: 0     0.00
Movie          0.00
Year           0.02
Genre          0.00
RunTime        0.17
Rating         0.13
Director       0.13
Actor          0.02
Description    0.00
IMDB_Score     0.12
Metascore      0.96
Votes          0.12
USA_Gross      0.96
Category       0.00
dtype: float64

- Cột Metascore, USA_Gross có tỉ lệ thiếu dữ liệu quá lớn (0.96), ta sẽ drop hai cột này.
- Cột IMDB_Score sẽ được sử dụng trong phân tích nên sẽ được giữ lại các vị trí trống, còn lại ta sẽ drop các hàng có giá trị thiếu ở những cột khác.
- Ngoài ra, ta cũng không cần cột index có sẵn của dataset.

## Tiền xử lý

In [5]:
mdi_df['IMDB_Score'] = mdi_df['IMDB_Score'].fillna(0)
mdi_df = mdi_df.drop(columns = ['Unnamed: 0', 'Metascore', 'USA_Gross'])
mdi_df = mdi_df.dropna(axis = 0).reset_index().drop(columns = 'index')

In [6]:
mdi_df.shape

(1247, 11)

In [7]:
mdi_df.isna().sum()

Movie          0
Year           0
Genre          0
RunTime        0
Rating         0
Director       0
Actor          0
Description    0
IMDB_Score     0
Votes          0
Category       0
dtype: int64

## Khám phá dữ liệu (tiếp tục)

In [8]:
mdi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1247 entries, 0 to 1246
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Movie        1247 non-null   object 
 1   Year         1247 non-null   object 
 2   Genre        1247 non-null   object 
 3   RunTime      1247 non-null   object 
 4   Rating       1247 non-null   object 
 5   Director     1247 non-null   object 
 6   Actor        1247 non-null   object 
 7   Description  1247 non-null   object 
 8   IMDB_Score   1247 non-null   float64
 9   Votes        1247 non-null   object 
 10  Category     1247 non-null   object 
dtypes: float64(1), object(10)
memory usage: 107.3+ KB


Xem xét các cột có dtype là Object, ta thấy định dạng ở một số cột có vẻ khá khó để thao tác, ta sẽ xử lý những cột này.

## Tiền xử lý (tiếp tục)

Bắt đầu xử lý với cột Year.

In [9]:
mdi_df['Year'].unique()

array(['(2019)', '(2014)', '(2017)', '(2018)', '(2015)', '(2011)',
       '(2008)', '(2016)', '(2012)', '(2013)', '(2010)', '(2007)',
       '(2017 Video)', '(2013 Video)', '(2016 Video)',
       '(2014 TV Special)', '(2009 Video)', '(2020 TV Special)',
       '(2006 Video)', '(2016 TV Special)', '(2015 TV Special)',
       '(2012 Video)', '(2011 Video)', '(2007 Video)',
       '(2018 TV Special)', '(2017 TV Special)', '(2014 Video)', '(2020)',
       '(2002 Video)', '(III) (2011)', '(2001–2004)', '(2000–2003)',
       '(2021)', '(2021– )', '(2015–2019)', '(2016–2018)', '(2013–2020)',
       '(2017–2019)', '(2009)', '(2002)', '(2005)', '(1989)', '(1997)',
       '(1995)', '(1978)', '(1992)', '(1984)', '(1999)', '(2006)',
       '(1994)', '(2003)', '(1983)', '(1980)', '(2004)', '(1982)',
       '(2008 Video)', '(2000 Video)', '(2015 Video)', '(1998 Video)',
       '(2005 Video)', '(2006 TV Movie)', '(1999 TV Movie)',
       '(2018 Video)', '(1984 TV Movie)', '(2001–2011)', '(1999–2001)'

Theo quan sát, cột Year dùng các dạng sau để thể hiện giá trị (với X, Y là năm và Z là các phần của phim):
- (X)
- (X-)
- (X Video)
- (X Video Game)
- (X TV Special)
- (X TV Movie)
- (X - Y)
- (Z)
- (Z) (X)

In [10]:
mdi_df['Year'] = mdi_df['Year'].apply(lambda x: x.replace('(','')\
                                      .replace(')', '')\
                                      .replace(' Video', '')\
                                      .replace(' TV Special','')\
                                      .replace('III ','')\
                                      .replace('–','')\
                                      .replace(' TV Movie',''))
mdi_df['Year'] = mdi_df['Year'].astype(int)
mdi_df['Year'].unique()

array([    2019,     2014,     2017,     2018,     2015,     2011,
           2008,     2016,     2012,     2013,     2010,     2007,
           2009,     2020,     2006,     2002, 20012004, 20002003,
           2021, 20152019, 20162018, 20132020, 20172019,     2005,
           1989,     1997,     1995,     1978,     1992,     1984,
           1999,     1994,     2003,     1983,     1980,     2004,
           1982,     2000,     1998, 20012011, 19992001, 19921995,
       20042008, 19971999, 20032005, 20042006, 19962000, 20032006,
       20012003, 20052006, 20062010, 20102011, 20122020, 20142019,
       20132014, 20142015, 20152021, 20152016, 20182019, 20172021])

Với cách xử lý này, những bộ phim được thực hiện trong nhiều năm sẽ có cách thể hiện hơi không đúng lắm, ta có thể chuyển cột Year thành 2 cột StartYear và EndYear.

In [11]:
def get_start_year(num):
    if num < 2100:
        return num
    else:
        return num // 10000
    
Start_Year =  mdi_df['Year'].apply(lambda x: get_start_year(x))
Start_Year.name = 'StartYear'
End_Year = mdi_df['Year'].apply(lambda x: x % 10000)
End_Year.name = 'EndYear'
mdi_df = mdi_df.drop(columns = ['Year'])

In [12]:
mdi_df = pd.concat([mdi_df, Start_Year, End_Year], axis = 1)
mdi_df.head()

Movie                    Genre  RunTime Rating  \
0          Avengers: Endgame   Action,Adventure,Drama  181 min  PG-13   
1    Guardians of the Galaxy  Action,Adventure,Comedy  121 min  PG-13   
2  Spider-Man: Far from Home  Action,Adventure,Sci-Fi  129 min  PG-13   
3             Thor: Ragnarok  Action,Adventure,Comedy  130 min  PG-13   
4     Avengers: Infinity War  Action,Adventure,Sci-Fi  149 min  PG-13   

                Director                                              Actor  \
0  AnthonyRusso,JoeRusso  RobertDowneyJr.,ChrisEvans,MarkRuffalo,ChrisHe...   
1              JamesGunn      ChrisPratt,VinDiesel,BradleyCooper,ZoeSaldana   
2               JonWatts  TomHolland,SamuelL.Jackson,JakeGyllenhaal,Mari...   
3           TaikaWaititi  ChrisHemsworth,TomHiddleston,CateBlanchett,Mar...   
4  AnthonyRusso,JoeRusso  RobertDowneyJr.,ChrisHemsworth,MarkRuffalo,Chr...   

                                         Description  IMDB_Score      Votes  \
0  After the devastating events of Avengers: Infi...         8.4    880,911   
1  A group of intergalactic criminals must pull t...         8.0  1,066,222   
2  Following the events of Avengers: Endgame (201...         7.5    348,047   
3  Imprisoned on the planet Sakaar, Thor must rac...         7.9    615,860   
4  The Avengers and their allies must be willing ...         8.4    881,638   

  Category  StartYear  EndYear  
0   Marvel       2019     2019  
1   Marvel       2014     2014  
2   Marvel       2019     2019  
3   Marvel       2017     2017  
4   Marvel       2018     2018

Tiếp theo ta sẽ xử lý cột RunTime, đơn vị 'min' sẽ được loại bỏ, giá trị thể hiện sẽ chuyển về dạng số.

In [13]:
mdi_df['RunTime'] = mdi_df['RunTime'].apply(lambda x: x.replace('min',''))
mdi_df['RunTime'] = mdi_df['RunTime'].astype(int)
mdi_df['RunTime']

0       181
1       121
2       129
3       130
4       149
       ... 
1242     42
1243     42
1244     42
1245     42
1246     42
Name: RunTime, Length: 1247, dtype: int32

Với cột Votes, giá trị được phân cách hàng nghìn bởi dấu ',' ta sẽ xử lý và đưa về dạng số.

In [14]:
mdi_df['Votes'] = mdi_df['Votes'].apply(lambda x: x.replace(',',''))
mdi_df['Votes'] = mdi_df['Votes'].astype(int)
mdi_df['Votes']

0        880911
1       1066222
2        348047
3        615860
4        881638
         ...   
1242       2050
1243       1259
1244       1053
1245       1036
1246       1037
Name: Votes, Length: 1247, dtype: int32

Dataframe sau xử lý:

In [15]:
mdi_df = mdi_df[['Movie', 'Genre', 'RunTime', 'StartYear', 'EndYear', 'Rating', \
                 'Director', 'Actor', 'Description', 'IMDB_Score', 'Votes', 'Category']]
mdi_df.head()

Movie                    Genre  RunTime  StartYear  \
0          Avengers: Endgame   Action,Adventure,Drama      181       2019   
1    Guardians of the Galaxy  Action,Adventure,Comedy      121       2014   
2  Spider-Man: Far from Home  Action,Adventure,Sci-Fi      129       2019   
3             Thor: Ragnarok  Action,Adventure,Comedy      130       2017   
4     Avengers: Infinity War  Action,Adventure,Sci-Fi      149       2018   

   EndYear Rating               Director  \
0     2019  PG-13  AnthonyRusso,JoeRusso   
1     2014  PG-13              JamesGunn   
2     2019  PG-13               JonWatts   
3     2017  PG-13           TaikaWaititi   
4     2018  PG-13  AnthonyRusso,JoeRusso   

                                               Actor  \
0  RobertDowneyJr.,ChrisEvans,MarkRuffalo,ChrisHe...   
1      ChrisPratt,VinDiesel,BradleyCooper,ZoeSaldana   
2  TomHolland,SamuelL.Jackson,JakeGyllenhaal,Mari...   
3  ChrisHemsworth,TomHiddleston,CateBlanchett,Mar...   
4  RobertDowneyJr.,ChrisHemsworth,MarkRuffalo,Chr...   

                                         Description  IMDB_Score    Votes  \
0  After the devastating events of Avengers: Infi...         8.4   880911   
1  A group of intergalactic criminals must pull t...         8.0  1066222   
2  Following the events of Avengers: Endgame (201...         7.5   348047   
3  Imprisoned on the planet Sakaar, Thor must rac...         7.9   615860   
4  The Avengers and their allies must be willing ...         8.4   881638   

  Category  
0   Marvel  
1   Marvel  
2   Marvel  
3   Marvel  
4   Marvel

## Phân tích dữ liệu để trả lời câu hỏi

Hiện tại, hai hãng phim muốn bắt đầu một dự án mới, tuy nhiên hãng cũng muốn đánh giá xem, đâu là thể loại phim được ưa chuộng nhất để lên ý tưởng, đạo diễn cũng như diễn viên cho bộ phim này là những ai để thu lại được kết quả tốt nhất. Chúng ta hãy cùng phân tích qua bộ dữ liệu này để tìm được những tiêu chí phù hợp cho hai hãng phim.

Về sơ bộ, ý tưởng của việc phân tích như sau:
- Một bộ phim được khán giả ưa thích (có lượt Votes cao) thì hẳn nhiên phần lớn các yếu tố của bộ phim cũng vậy. Thể hiện khái quát nhất của điều này nằm ở thể loại phim. Như vậy, ta có thể tìm được thể loại phim được ưa thích nhất qua tổng hợp các lượt Votes.
- Đạo diễn cũng như diễn viên phù hợp nhất cho bộ phim có tham vọng hẳn nhiên phải là những người đã từng thực hiện và cho ra những bộ phim được đánh giá cao ở thể loại phim đó. Như vậy, từ thể loại phim đã chọn ở trên, ta sẽ chọn ra những đạo diễn và diễn viên đã từng tham gia những bộ phim có thể loại này và được cho giới chuyên môn đánh giá cao (IMDB_Score cao).

Lưu ý, để không làm phức tạp thêm bài toán, ta sẽ tạm bỏ qua các yếu tố như: 
- Những người được chọn liệu có đang bận thực hiện một bộ phim khác hay không?
- Những người được chọn đóng những bộ phim cho lứa tuổi nào?
- Cũng như việc lượt Votes chưa thực sự thể hiện hết lượng người yêu thích bộ phim.

### Xếp hạng các thể loại phim được ưa chuộng nhất

In [16]:
df = mdi_df[['Genre', 'Votes']]
df

Genre    Votes
0      Action,Adventure,Drama   880911
1     Action,Adventure,Comedy  1066222
2     Action,Adventure,Sci-Fi   348047
3     Action,Adventure,Comedy   615860
4     Action,Adventure,Sci-Fi   881638
...                       ...      ...
1242   Action,Adventure,Drama     2050
1243   Action,Adventure,Drama     1259
1244   Action,Adventure,Drama     1053
1245   Action,Adventure,Drama     1036
1246   Action,Adventure,Drama     1037

[1247 rows x 2 columns]

In [17]:
df['Genre'] = df['Genre'].str.split(',')
df = df.explode('Genre')
df.groupby('Genre').sum().sort_values(by = ['Votes'], ascending = False).head()

<ipython-input-17-edce1078b9b2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Genre'] = df['Genre'].str.split(',')


Votes
Genre              
Action     32017312
Adventure  26174909
Sci-Fi     13929388
Drama       9338947
Crime       5360704

Dựa vào kết quả trên, bộ phim mới sẽ có sự kết hợp giữa các thể loại : Action, Adventure và Sci-Fi.

### Chọn ra đạo diễn cho bộ phim

In [18]:
df = mdi_df[['Genre', 'Director', 'IMDB_Score']]
df

Genre               Director  IMDB_Score
0      Action,Adventure,Drama  AnthonyRusso,JoeRusso         8.4
1     Action,Adventure,Comedy              JamesGunn         8.0
2     Action,Adventure,Sci-Fi               JonWatts         7.5
3     Action,Adventure,Comedy           TaikaWaititi         7.9
4     Action,Adventure,Sci-Fi  AnthonyRusso,JoeRusso         8.4
...                       ...                    ...         ...
1242   Action,Adventure,Drama           GregorySmith         8.5
1243   Action,Adventure,Drama             CarlSeaton         8.3
1244   Action,Adventure,Drama        AlexisOstrander         8.1
1245   Action,Adventure,Drama           ShannonKohli         7.4
1246   Action,Adventure,Drama       BenHernandezBray         7.5

[1247 rows x 3 columns]

In [19]:
df['Genre'] = df['Genre'].str.split(',')
df = df.explode('Genre')
df

<ipython-input-19-3eaa99576f91>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Genre'] = df['Genre'].str.split(',')


Genre               Director  IMDB_Score
0        Action  AnthonyRusso,JoeRusso         8.4
0     Adventure  AnthonyRusso,JoeRusso         8.4
0         Drama  AnthonyRusso,JoeRusso         8.4
1        Action              JamesGunn         8.0
1     Adventure              JamesGunn         8.0
...         ...                    ...         ...
1245  Adventure           ShannonKohli         7.4
1245      Drama           ShannonKohli         7.4
1246     Action       BenHernandezBray         7.5
1246  Adventure       BenHernandezBray         7.5
1246      Drama       BenHernandezBray         7.5

[3707 rows x 3 columns]

In [20]:
df['Director'] = df['Director'].str.split(',')
df = df.explode('Director')
df

Genre          Director  IMDB_Score
0        Action      AnthonyRusso         8.4
0        Action          JoeRusso         8.4
0     Adventure      AnthonyRusso         8.4
0     Adventure          JoeRusso         8.4
0         Drama      AnthonyRusso         8.4
...         ...               ...         ...
1245  Adventure      ShannonKohli         7.4
1245      Drama      ShannonKohli         7.4
1246     Action  BenHernandezBray         7.5
1246  Adventure  BenHernandezBray         7.5
1246      Drama  BenHernandezBray         7.5

[3902 rows x 3 columns]

In [21]:
genre_lists = ['Action', 'Adventure', 'Sci-Fi']

t_df = df[df['Genre'].isin(genre_lists)]
#t_df = t_df.groupby(['Genre','Director']).mean().sort_values(by = ['IMDB_Score'], ascending = False)
t_df

Genre          Director  IMDB_Score
0        Action      AnthonyRusso         8.4
0        Action          JoeRusso         8.4
0     Adventure      AnthonyRusso         8.4
0     Adventure          JoeRusso         8.4
1        Action         JamesGunn         8.0
...         ...               ...         ...
1244  Adventure   AlexisOstrander         8.1
1245     Action      ShannonKohli         7.4
1245  Adventure      ShannonKohli         7.4
1246     Action  BenHernandezBray         7.5
1246  Adventure  BenHernandezBray         7.5

[2100 rows x 3 columns]